## Importing the necessary libraries 

In [3]:
import pandas as pd
import numpy as np
from textstat.textstat import textstat
import sys
import os
import string
import math
import random
import re
from sklearn.metrics import make_scorer
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import *

In [2]:
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

#### Adding the liblinear path

#### Importing the classifier

#### Importing sklearn libraries

In [ ]:
from sklearn.metrics import log_loss
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.grid_search import RandomizedSearchCV
from sklearn.preprocessing import FunctionTransformer

from sklearn.dummy import DummyClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import NearestNeighbors
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import svm
from sklearn.neural_network import MLPClassifier

## Stating the path of the data

#### Given Dataset

In [4]:
train_path = "../data/data/train.csv"
test_path = "../data/data/test.csv"

#### Additional Data

In [5]:
emotion_lexicon_path = "../data/features/NRC-AffectIntensity-Lexicon.txt"

In [6]:
regex_image = re.compile("((\S*)|((Image:|File:).*))(.gif|.png|.tiff|.jpg|.jpeg)", re.IGNORECASE)

In [7]:
regex_links = re.compile("((http)|(www))\S*", re.IGNORECASE)

In [8]:
regex_new_line = re.compile("\n", re.IGNORECASE)

In [9]:
regex_multiple_white_space = re.compile("\s+", re.IGNORECASE)

In [48]:
regex_users = re.compile("[0-9]{1,3}\.[0-9]{1,3}\.[0-9]{1,3}\.[0-9]{1,3}", re.IGNORECASE)

In [60]:
regex_user1 = re.compile("[0-9]{1,3}\.[0-9]{1,3}\.[0-9]{1,3}\.[0-9]{1,3}", re.IGNORECASE)
regex_user2 = re.compile("\[{1,2}(User:|User Talk:|User talk:|User_talk:)\s?\w*(\||\]{1,2})?", re.IGNORECASE)

regex_list_user = [regex_user1, regex_user2]

def replace_user_multiple_regex(comment, regexList):
    for regex in regexList:
        comment = regex.sub("<user>", comment)
    return comment

### Defining helper functions

In [17]:
def remove_new_line(comment, regex):
    comment = regex.sub(" ", comment)
    return comment

In [18]:
def replace_image(comment, regex):
    comment = regex.sub("<image>", comment)
    return comment

In [19]:
def replace_links(comment, regex):
    comment = regex.sub("<link>", comment)
    return comment

In [20]:
def remove_mutiple_white_space(comment, regex):
    comment = regex.sub(" ", comment).strip()
    return comment

In [21]:
def replace_user(comment, regex):
    comment = regex.sub("<user>", comment)
    return comment

In [64]:
#for index in [8419, 8417, 3695]:
for index in [3695]:
    trial = trainData.iloc[index,1]
    print(trial)
    print("-----------------------------------------------------------------------------------------------")
    trial = remove_new_line(trial, regex_new_line)
    trial = remove_mutiple_white_space(trial, regex_multiple_white_space)
    trial = replace_links(trial, regex_links)
    trial = replace_user_multiple_regex(trial, regex_list_user)
    print(trial)

"

 Edit war brewing at Yosemite National Park 

If you have some spare energy, please come and lend a hand. Thanks! —  

Looked ... acted ... )   

I was also approached recently with regards to possible edits to the Yosemite National Park article. My input on the matter can be found at the following link;
user talk:Edit Centric#Hantavirus Risk in Yosemite

Noted,   

 Thmc1 / 173.63.176.93 sill sockpuppeting / REPLYU REQUSTED 

This person (blocked sockmaster Thmc1) is still block evading by not logging in, even after he was warned by SysAdmin Kudpung. I would suggest that you block the suspected IPs(173.63176.93, 74.88.160.244) as well as IPs used by his other blocked account (Nyc88) and any other accounts affiliated w/them. Why are sockppuppets of a blocked sockmaster still allowed to edit even after ample warnings? To make matters worse, he even violated his block and continuted editing the day after he was blocked! Please see Thmc1 sockpuppet investigation archive for latest inve

In [15]:
trainData = pd.read_csv(train_path)
trainData.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,22256635,"Nonsense? kiss off, geek. what I said is true...",1,0,0,0,0,0
1,27450690,"""\n\n Please do not vandalize pages, as you di...",0,0,0,0,0,0
2,54037174,"""\n\n """"Points of interest"""" \n\nI removed the...",0,0,0,0,0,0
3,77493077,Asking some his nationality is a Racial offenc...,0,0,0,0,0,0
4,79357270,The reader here is not going by my say so for ...,0,0,0,0,0,0


In [22]:
trial1 = trainData.iloc[67,1]
trial1

'Image:John Holmes arrested.jpg listed for deletion \nAn image or media file that you uploaded or altered, Image:John Holmes arrested.jpg, has been listed at Wikipedia:Images and media for deletion. Please see the discussion to see why this is (you may have to search for the title of the image to find its entry), if you are interested in it not being deleted.  —gr'

#### Data Cleaning Methods

In [ ]:
def clean_comment(comment): # cleaning to be filled up
    if math.isnan(comment):
        return ""
    else:
        return str(comment)

### Textstat Features

In [ ]:
def average_syllable(comment):
    comment_lst = comment.split(" ")
    syllable_count = []
    for word in comment_lst:
        try:
            syllable_count.extend([textstat.syllable_count(word)])
        except:
            continue
    
    if len(syllable_count) == 0:
        return 0.0
    else:
        return np.average(syllable_count)

### Emotion Features

#### Simple Translator to remove all puncutuations 

In [ ]:
translator = str.maketrans('', '', string.punctuation)

#### To get the average emotion score

In [ ]:
def avg_emotion(comment, dict_):
    try:
        comment = comment.translate(translator)
    except:
        comment = ""
    comment_lst = comment.split(" ")
    score_lst = [dict_.get(word.lower()) for word in comment_lst if dict_.get(word.lower()) is not None]
    if len(score_lst) == 0:
        return 0.0
    else:
        return np.average(score_lst)

### Boolean Features

In [5]:
def check_file(comment):
    extensions = [".jpg", ".png", ".jpeg", ".gif", "tiff"]
    
    for extension in extensions: 
        
        if extension in comment.lower():
            return 1
    return 0

### Defining the Classes 

#### Data File Class

In [ ]:
class DataFile:
    
    def __init__(self, path):
        self.__path = path
        df = pd.read_csv(path).replace(np.nan, '', regex=True)
        self.__df = df
        
    def dataCleaning(self):
        self.__df.loc[:, "comment_text"] = self.__df.loc[:, "comment_text"].apply(lambda x : clean_comment(x) )
    
    def getDF(self):
        return self.__df
    
    def updateDF(self, new_df):
        self.__df = new_df
    

#### Feature Engineering Class 

In [ ]:
class FeaturesEngineer:
    
    def __init__(self, emotion_lexicon_path):
        self.emotion_lexicon_path = emotion_lexicon_path
        self.emotion_lexicon = pd.read_csv(self.emotion_lexicon_path, sep = "\t")
        
        self.anger = self.emotion_lexicon.loc[self.emotion_lexicon.loc[:, "AffectDimension"] == "anger", :]
        self.fear = self.emotion_lexicon.loc[self.emotion_lexicon.loc[:, "AffectDimension"] == "fear", :]
        self.joy = self.emotion_lexicon.loc[self.emotion_lexicon.loc[:, "AffectDimension"] == "joy", :]
        self.sadness = self.emotion_lexicon.loc[self.emotion_lexicon.loc[:, "AffectDimension"] == "sadness", :]
        
        anger_dict = {}
        for i in range(0, self.anger.shape[0]):
            word = self.anger.iloc[i, :].term
            score = self.anger.iloc[i, :].score
            anger_dict[word] = score
            
        fear_dict = {}
        for i in range(0, self.fear.shape[0]):
            word = self.fear.iloc[i, :].term
            score = self.fear.iloc[i, :].score
            fear_dict[word] = score
            
        joy_dict = {}
        for i in range(0, self.joy.shape[0]):
            word = self.joy.iloc[i, :].term
            score = self.joy.iloc[i, :].score
            joy_dict[word] = score
            
        sadness_dict = {}
        for i in range(0, self.sadness.shape[0]):
            word = self.sadness.iloc[i, :].term
            score = self.sadness.iloc[i, :].score
            sadness_dict[word] = score
            
        self.anger_dict = anger_dict
        self.fear_dict = fear_dict
        self.joy_dict = joy_dict
        self.sadness_dict = sadness_dict
    
    def createTextStatFeatures(self, df):
        
        df.loc[:,'comment_len'] = df.loc[:,'comment_text'].apply(lambda x: len(x))
        df.loc[:,'comment_avg_syllable'] = df.loc[:,'comment_text'].apply(lambda x: average_syllable(x))
        df.loc[:,'comment_syllable'] = df.loc[:,'comment_text'].apply(lambda x: textstat.syllable_count(x))
        df.loc[:,'comment_num_sent'] = df.loc[:, 'comment_text'].apply(lambda x: textstat.sentence_count(x))
        df.loc[:, "comment_word_per_sent"] = df.loc[:, "comment_text"].apply(lambda x: textstat.lexicon_count(x) / textstat.sentence_count(x))
        df.loc[:,'comment_flesch_reading_ease'] = df.loc[:,'comment_text'].apply(lambda x: textstat.flesch_reading_ease(x))
        df.loc[:,'comment_flesch_kincaid_grade'] = df.loc[:,'comment_text'].apply(lambda x: textstat.flesch_kincaid_grade(x))
        
        return df
        
    def createEmotionFeatures(self, df):
        
        df.loc[:, "avg_anger"] = df.loc[:, "comment_text"].apply(lambda x: avg_emotion(x, self.anger_dict))
        df.loc[:, "avg_fear"] = df.loc[:, "comment_text"].apply(lambda x: avg_emotion(x, self.fear_dict))
        df.loc[:, "avg_joy"] = df.loc[:, "comment_text"].apply(lambda x: avg_emotion(x, self.joy_dict))
        df.loc[:, "avg_sadness"] = df.loc[:, "comment_text"].apply(lambda x: avg_emotion(x, self.sadness_dict))
        
        return df
    
    def booleanFeatures()
    
    def intensifiersFeatures(self, df):
        
        df.loc[:, "emphasized_words"] = df.loc[:, "comment_text"].apply(lambda x: x)


### Defining the Evaluation Metric

In [ ]:
def loss_func(y_label, y_proba):
    return log_loss(y_label, y_proba)

### Classification pipeline

#### Function to get the engineered features 

In [ ]:
def get_manual_features(df):
    
    col = ["id", "comment_text", "toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate" ]
    df_sieved = df.drop(col, axis = 1)
    
    return df_sieved
    

#### Turning to FT

In [ ]:
get_manual_features_ft = FunctionTransformer(get_manual_features, validate = False)

### To do grid search for the best C 

#### Splitting the data in nr-fold cv 

In [ ]:
def split_data(num, cv):
    
    num_points = int(num/cv)
    num_points_last = num - (cv-1) * num_points
    selected = []
    index_test = []
    index_train = []
    
    for i in range(0, cv):
        if i == cv-1:
            choices = [x for x in range(0, num) if x not in selected]
            picks = random.sample(choices, num_points_last)
            remaining = [x for x in range(0, num) if x not in picks]
            index_test.append(picks)
            index_train.append(remaining)
            selected.extend(picks)
        else:
            choices = [x for x in range(0, num) if x not in selected]
            picks = random.sample(choices, num_points)
            remaining = [x for x in range(0, num) if x not in picks]
            index_test.append(picks)
            index_train.append(remaining)
            selected.extend(picks)
            
    return index_train, index_test

#### Getting the best C

In [ ]:
def best_C(min_c, max_c, step, data, label, cv):
    index_train, index_test = split_data(data.shape[0], cv)
    c_range = np.arange(min_c, max_c, step)
    loss_lst_compiled = []
    for c in c_range:
        val = 2.0 ** c
        loss_lst = []
        for i in range(0, cv):
            test_ = index_test[i]
            train_ = index_train[i]
            
            features_train = data.iloc[train_, :]
            labels_train = data.iloc[train_, :].loc[:, label]
            
            features_test = data.iloc[test_, :]
            labels_test = data.iloc[test_, :].loc[:, label]
            
            param = "-c " + str(val) + " -s " + str(self.algo)
    
            pipe.fit(features_train, labels_train)
            labels = pipe.predict(features_test)
            proba = pipe.predict_proba(features_test)
            
            loss = loss_func(labels_test, proba)
            loss_lst.extend([loss])
        
        avg_loss = np.average(loss_lst)
        loss_lst_compiled.append(avg_loss)
        
    loss_lst_compiled = [x for x in zip(c_range, loss_lst_compiled)]  
    loss_lst_compiled = sorted(loss_lst_compiled, key= lambda x: x[1])
    return  loss_lst_compiled[0]


### Defining the main function 

In [ ]:
def main():
    
    labels = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
    loss_compiled = []
    
    for label in labels:
        best_c, best_rate = best_C(-5,5,1, trainData.getDF(), label, 5)
        print('Log loss for "{}" based on 5 fold CV: {}, with log2c = {}'.format(label, best_rate, best_c))
        loss_compiled.extend([best_rate])
    
    print(loss_compiled)
    avg_loss = np.average(loss_compiled)
    print('Average Log loss = {}'.format(avg_loss))
    

#### Running the main function 

In [ ]:
if __name__ == "__main__":
    
    #extractor = FeaturesEngineer(emotion_lexicon_path)
    #trainData = DataFile(path = train_path)
    print("Done reading in train data")
    #testData = DataFile(path= test_path)
    #print("Done reading in test data")
    
    #trainData.updateDF(extractor.createEmotionFeatures(trainData.getDF()))
    print("Done creating emotion features for train data")
    #testData.updateDF(extractor.createEmotionFeatures(testData.getDF()))
    #print("Done creating emotion features for test data")
    
    #trainData.updateDF(extractor.createTextStatFeatures(trainData.getDF()))
    print("Done creating textstat features for train data")
    #testData.updateDF(extractor.createTextStatFeatures(testData.getDF()))
    #print("Done creating textstat features for test data")
    
    main()
    

In [ ]:
trainData.getDF().to_csv("../data/features/train_data_cache.csv", index = False)

In [ ]:
LogisticRegression(max_iter=)